In [2]:
"""
交互式太阳系行星模拟器
作者：[你的名字]
日期：2023年11月
描述：基于牛顿万有引力定律的太阳系行星轨道模拟
"""

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, Checkbox, VBox, HBox, Label
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

class SolarSystemSimulator:
    """太阳系行星模拟器类"""
    
    def __init__(self):
        # 行星数据（名称，颜色，半径比例，轨道半径比例，轨道周期比例）
        self.planets_data = [
            ("太阳", "yellow", 20, 0, 0),      # 太阳（中心）
            ("水星", "gray", 2, 10, 0.24),     # 轨道半径AU，轨道周期年
            ("金星", "orange", 4, 15, 0.62),
            ("地球", "blue", 4, 20, 1),
            ("火星", "red", 3, 25, 1.88),
            ("木星", "brown", 8, 40, 11.86),
            ("土星", "gold", 7, 55, 29.46),
            ("天王星", "cyan", 6, 70, 84.01),
            ("海王星", "darkblue", 6, 85, 164.8)
        ]
        
        # 初始化参数
        self.time = 0
        self.time_step = 0.02
        self.view_mode = "2D"
        self.show_orbits = True
        self.show_physics = True
        
        # 创建图形
        self.fig = plt.figure(figsize=(14, 8))
        
    def calculate_positions(self, time):
        """计算行星在给定时间的位置"""
        positions = []
        for i, (name, color, size, radius, period) in enumerate(self.planets_data):
            if i == 0:  # 太阳固定在中心
                positions.append((0, 0, 0))
            else:
                # 基于开普勒定律计算位置
                # 角速度 = 2π / 周期
                angular_velocity = 2 * np.pi / (period if period > 0 else 1)
                angle = angular_velocity * time
                
                # 实际物理中，行星轨道是椭圆，这里简化为圆形
                x = radius * np.cos(angle)
                y = radius * np.sin(angle)
                z = 0
                
                # 添加一些轨道倾角，使3D视图更有趣
                if self.view_mode == "3D":
                    inclination = (i-1) * 5 * np.pi/180  # 小倾角
                    y = y * np.cos(inclination)
                    z = radius * 0.1 * np.sin(angle + i)  # 小起伏
                    
                positions.append((x, y, z))
        return positions
    
    def calculate_physics(self):
        """计算和显示物理参数"""
        # 简化的物理计算
        positions = self.calculate_positions(self.time)
        
        # 计算地球的轨道速度（近似）
        earth_index = 3  # 地球
        sun_pos = positions[0]
        earth_pos = positions[earth_index]
        
        # 地球到太阳的距离
        distance = np.sqrt((earth_pos[0]-sun_pos[0])**2 + 
                          (earth_pos[1]-sun_pos[1])**2)
        
        # 根据开普勒第三定律计算轨道速度
        # T² ∝ r³，v = 2πr/T
        orbital_period = self.planets_data[earth_index][4]  # 年
        orbital_speed = 2 * np.pi * distance / orbital_period
        
        return distance, orbital_speed
    
    def update_plot(self, time_speed=1.0, view_mode="2D", 
                    show_orbits=True, show_physics=True,
                    gravity_constant=1.0):
        """更新绘图"""
        self.time += self.time_step * time_speed
        self.view_mode = view_mode
        self.show_orbits = show_orbits
        self.show_physics = show_physics
        
        # 清除之前的图形
        self.fig.clf()
        
        # 根据视图模式设置子图
        if view_mode == "2D":
            ax = self.fig.add_subplot(111)
            ax.set_aspect('equal')
            
            # 设置坐标轴范围
            max_radius = 90
            ax.set_xlim(-max_radius, max_radius)
            ax.set_ylim(-max_radius, max_radius)
            
            # 绘制背景网格
            ax.grid(True, alpha=0.3, linestyle='--')
            
            # 绘制太阳
            ax.scatter(0, 0, color='yellow', s=300, edgecolors='orange', 
                      linewidths=2, zorder=10)
            ax.text(0, 3, '太阳', ha='center', fontsize=12, fontweight='bold')
            
        else:  # 3D视图
            ax = self.fig.add_subplot(111, projection='3d')
            
            # 设置坐标轴范围
            max_radius = 90
            ax.set_xlim(-max_radius, max_radius)
            ax.set_ylim(-max_radius, max_radius)
            ax.set_zlim(-max_radius/3, max_radius/3)
            
            ax.set_xlabel('X (天文单位)')
            ax.set_ylabel('Y (天文单位)')
            ax.set_zlabel('Z (天文单位)')
        
        # 计算并绘制行星位置
        positions = self.calculate_positions(self.time)
        
        # 存储行星路径（用于绘制轨道）
        if not hasattr(self, 'planet_paths'):
            self.planet_paths = [[] for _ in range(len(self.planets_data))]
        
        # 绘制每个行星
        for i, (name, color, size, radius, period) in enumerate(self.planets_data):
            if i == 0:  # 太阳已经绘制
                continue
                
            x, y, z = positions[i]
            
            # 记录路径
            self.planet_paths[i].append((x, y, z))
            
            # 限制路径长度
            if len(self.planet_paths[i]) > 200:
                self.planet_paths[i].pop(0)
            
            # 绘制轨道（如果启用）
            if show_orbits and len(self.planet_paths[i]) > 1:
                path = np.array(self.planet_paths[i])
                if view_mode == "2D":
                    ax.plot(path[:, 0], path[:, 1], color=color, 
                           alpha=0.3, linewidth=1)
                else:
                    ax.plot(path[:, 0], path[:, 1], path[:, 2], 
                           color=color, alpha=0.3, linewidth=1)
            
            # 绘制行星
            if view_mode == "2D":
                ax.scatter(x, y, color=color, s=size*15, edgecolors='white', 
                          linewidths=1, zorder=5)
                ax.text(x, y+size/2, name, ha='center', fontsize=9)
            else:
                ax.scatter(x, y, z, color=color, s=size*20, 
                          edgecolors='white', linewidths=1)
                ax.text(x, y, z+size/2, name, ha='center', fontsize=9)
            
            # 绘制到太阳的连线（显示引力）
            if i <= 4:  # 只显示内行星的引力线
                if view_mode == "2D":
                    ax.plot([0, x], [0, y], color=color, alpha=0.2, 
                           linestyle='--', linewidth=0.5)
        
        # 添加标题
        title = f"太阳系行星模拟器\n"
        title += f"时间: {self.time:.1f} 年 | "
        
        # 计算并显示物理参数
        if show_physics:
            distance, speed = self.calculate_physics()
            title += f"地日距离: {distance:.1f} AU | "
            title += f"地球轨道速度: {speed:.2f} AU/年"
            
            # 在图中添加物理原理说明
            physics_text = "物理原理:\n"
            physics_text += "1. 开普勒第一定律: 行星绕太阳作椭圆运动\n"
            physics_text += "2. 开普勒第二定律: 单位时间扫过面积相等\n"
            physics_text += "3. 开普勒第三定律: 周期² ∝ 半长轴³\n"
            physics_text += f"4. 引力常数: {gravity_constant:.2f}G"
            
            if view_mode == "2D":
                ax.text(-85, -80, physics_text, fontsize=9, 
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
            else:
                ax.text2D(0.02, 0.02, physics_text, transform=ax.transAxes, 
                         fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        ax.set_title(title, fontsize=14, fontweight='bold')
        
        # 添加坐标轴标签
        if view_mode == "2D":
            ax.set_xlabel('X (天文单位 AU)')
            ax.set_ylabel('Y (天文单位 AU)')
        else:
            ax.set_xlabel('X (AU)')
            ax.set_ylabel('Y (AU)')
            ax.set_zlabel('Z (AU)')
        
        plt.tight_layout()
        plt.show()
        
        # 在控制台输出当前状态
        print(f"模拟时间: {self.time:.2f} 年")
        if show_physics:
            print(f"地球轨道半径: {distance:.2f} AU")
            print(f"地球轨道速度: {speed:.2f} AU/年")
        print("-" * 50)

    def create_interactive_app(self):
        """创建交互式应用界面"""
        # 创建控件
        time_slider = FloatSlider(
            value=1.0, min=0.1, max=5.0, step=0.1,
            description='时间速度:', continuous_update=True
        )
        
        gravity_slider = FloatSlider(
            value=1.0, min=0.5, max=2.0, step=0.1,
            description='引力常数:', continuous_update=True
        )
        
        view_dropdown = Dropdown(
            options=['2D', '3D'],
            value='2D',
            description='视图模式:'
        )
        
        orbits_checkbox = Checkbox(
            value=True,
            description='显示轨道'
        )
        
        physics_checkbox = Checkbox(
            value=True,
            description='显示物理参数'
        )
        
        reset_button = Checkbox(
            value=False,
            description='重置模拟'
        )
        
        # 创建布局
        title = Label(value="🌌 交互式太阳系行星模拟器")
        subtitle = Label(value="调整参数探索轨道力学和开普勒定律")
        
        controls = VBox([
            title,
            subtitle,
            HBox([time_slider, gravity_slider]),
            HBox([view_dropdown, orbits_checkbox]),
            HBox([physics_checkbox, reset_button])
        ])
        
        # 定义交互函数
        def interactive_update(time_speed=1.0, gravity_constant=1.0,
                              view_mode="2D", show_orbits=True,
                              show_physics=True, reset=False):
            if reset:
                self.time = 0
                self.planet_paths = [[] for _ in range(len(self.planets_data))]
                reset_button.value = False
                
            self.update_plot(time_speed, view_mode, show_orbits, 
                           show_physics, gravity_constant)
        
        # 创建交互
        interact(interactive_update,
                time_speed=time_slider,
                gravity_constant=gravity_slider,
                view_mode=view_dropdown,
                show_orbits=orbits_checkbox,
                show_physics=physics_checkbox,
                reset=reset_button)
        
        return controls

# 运行应用
if __name__ == "__main__":
    # 创建模拟器实例
    simulator = SolarSystemSimulator()
    
    # 显示初始状态
    print("="*60)
    print("交互式太阳系行星模拟器")
    print("="*60)
    print("使用方法:")
    print("1. 调整'时间速度'控制模拟快慢")
    print("2. 选择'视图模式'切换2D/3D视图")
    print("3. 调整'引力常数'观察轨道变化")
    print("4. 勾选'显示轨道'查看行星轨迹")
    print("5. 勾选'显示物理参数'查看实时数据")
    print("6. 勾选'重置模拟'重新开始")
    print("="*60)
    
    # 创建交互界面
    controls = simulator.create_interactive_app()
    display(controls)
    
    # 显示初始图
    simulator.update_plot()

ModuleNotFoundError: No module named 'numpy'